# Assembly steps

With Shotgun-metatranscriptome-sequencing data \
Conda environment name: Assembly

- primarly for the illumina runs, paired-end sequences
- Steps are: 
    - QC with trimmomatic
        - bash script: Trimmomatic_QC.sh
    - PHIX filtering with bbmap
    - error correction with tedpole (from bbmap package)
    - assembly with metaSPAdes

## Long-read sequencing data (e.g. PacBio) 
Conda environment name: PacBio_Assembly

- for long read sequencing data from PacBio sequel platform: 
    - SRR10983239
    - SRR10983240
    - SRR10983241
    - SRR10983242
    
- Workflow:
    - QC using fastqc (results in RAWDATA/PacBio_runs/QC_metrics) and filtlong
    - Assembly using wtdbg2, metaFlye, Canu and  Trycycler
    - QC of assemblies with the metaQUAST

In [ ]:
module load gcc12-env/12.3.0
module load miniconda3/24.11.1

cd $WORK/DATA
mkdir QCed_DATA PHIX_FILTERED ERROR_CORRECTED ASSEMBLIES

### Progress Illumina

#### Trimmomatic PE: 
- Completed successfully
- Bigger files took longer and were interrupted often 
    - -> set longer time for the individual batch (max 48h) or launch parallel jobs

#### PhiX174 Removal:
- What ist PhiX?
    - PhiX control v3 library is smal library for quality control
    - Calibration control: phasing calculation, cross talk matrix generation, overall performance evaluation
    - Run quality monitoring: due to its balanced nucleotide composition
    - But should be removed to assemble the contigs
- Almost 100 % of the data wasn't mapped to PhiX, so the run was quite short
    - But there were files with higher error rate. (SRR15145662, SRR23378604, SRR23378605)

#### Error correction 
- with tadpole.sh of BBMap, correction mode
    - using given kmer counts and different algorithms
    - with rollback=t option, contigs with higher coverages are obtained
    - correct erros in read without extending or assemble them 
- Ran successfully
- Rate of reads with errors mostly > 80%
- No reduction in reads and bases
- Correction rate mostly under 2 % of total errors detected

#### Assembly
- Assemble the reads to contigs with different k-mers, metagenome assembly mode
- Run successful
    - But don't forget to set the sbatch time limit to max (2-00:00), otherwise it might be quitted haha
    - I had to rerun assembly for those canceled due to time limit
        - SRR15145660
        - SRR15145661
        - SRR15145662
        - SRR15145663
        - SRR15145664
        - SRR15145666
        - SRR23378604
        - SRR23378605
        - SRR6667231

#### QC
- FastQC and MultiQC
    - For Raw reads, QCed reads, PHIX removed reads, error corrected reads
    - also for EC reads mapped to the min500bp, renamed, prokaryotic contigs. 


### Progress PacBio
Assembled MAG from PacBio sequences in DATA/PacBio_Assembly directory
#### Filtlong
- QC and filtering
- But generally every reads are more than 1 kb long

#### Canu
- Error correction, trimming and assembly in one pipeline
    - error corrected files can also be used for other assembly pipelines like wtdbg2
- Runs were successful
    - check whether circular genomes are present 
        - look at the files with grep function
    - but: found no circular genome found 
        - Try with ASG metagenomic raw data?

 


In [ ]:
grep "^>" ASSEMBLIES/Canu/SRR10983239/SRR10983239.unitigs.fasta | head -n 20
grep "suggestCircular=yes" ASSEMBLIES/Canu/SRR10983242/SRR10983242.unitigs.fasta | head -n 20

#### wtdbg2
- runs successful but used not trimmed reads
- reran with trimmed reads (wtdbg_alt)
    - run successful
    - It would be interesting to see how different the results are
 

 #### metaFlye
 - assumes that reads are uncorrected
 - run complete

 #### Trycycler
- Takes too long, so try if you have time to spare

### Long read assembly QC
- metaQUAST for metagenomic MAG from long-read sequences with comparison of multiple assembly pipelines
- Results: 
    - No quality assessment possible for contigs from Canu and wtdbg2 (with EC reads) due to too low contig number (2-digits for Canu, 3 digits for wtdbg2 with ECed reads)
    - wtdbg2 contigs with raw reads (not ECed) has good balance of QC parameter and contig number/total length
    - MetaFlye with higher mismatch rate, duplication ratio, indels and error rate but much higher contigs number (4-digits) and more hits to references
- TO-DO: rerun with BUSCO database for bacteria for better results
- Only wtdbg2 (non-EC reads) contigs are used for further MAG assembly and analysis


    - check whether the sample contains no circular genome or the pipeline is not accurate: 
        - a PacBio raw read file from ASG database (ERR13615510) downloaded and ran through the canu pipeline

### Quality control
Quality assessment of reads of raw reads and after each processing steps
Tool: FastQC & multiQC
- FastQC
    - takes fastq file and generate summary graphs and tables + summary html
- MultiQC
    - combine the results of the outputs of FastQC
- Filtlong

- MultiQC_summary reports for each steps are created in html format
    - But not every samples are displayed in status checks hitmap plot
- Results
    - FastQC: Examples (for Illumina and PacBio): https://www.bioinformatics.babraham.ac.uk/projects/fastqc/    





In [ ]:
# for interactive interface: instead of sbatch run it just on the 
srun --pty --x11 --mem=10G --partition=interactive --time=2-00:00 

### Pipelines 
#### Canu
![Canu pipeline workflow image](Assembly/Pics/canu_pipeline.svg)

### DEBUGGING
#### Warning in slurm output file after running trimmomatic:
Exception in thread "Thread-0" java.lang.RuntimeException: Sequence and quality length don't match: 'AAAGGCATCGGTTGTCAGGTTCGTTGCTATGGGTGTACGTAACCATTTCAATTTTTCGGGGCGTAGGGCTTAGTTTCCACGCGTTCAATGAGAGCAGACCACCGTGAAGAGCTTTGGAATAGCACTTCAGTGGCTTCTGGCCATGACCTCC' vs 'F:FFFFF,FFF,F:FFFFF,FFFFFFFFFFFFFFFFFFFFFFFF::F:FFFFFFF:FFFFFFFFFFFFF:FFF,FFFFFFFFFFFFFFFFFFFFFF,FFF:F:F,FFFF,:F:FFFFF,FFF::FF,,F,:FF:,,FF:FF,:F:FFF,F,:F,FF,:FFFFFFFFF:F:,FFFF,,FFF,:,,,F,,:F:F:::,,FF,:F,FF,:F:FFF:FFFFFF:FFFFFFFFFFFFFF'
	at org.usadellab.trimmomatic.fastq.FastqRecord.<init>(FastqRecord.java:25)
	at org.usadellab.trimmomatic.fastq.FastqParser.parseOne(FastqParser.java:89)
	at org.usadellab.trimmomatic.fastq.FastqParser.next(FastqParser.java:179)
	at org.usadellab.trimmomatic.threading.ParserWorker.run(ParserWorker.java:42)
	at java.base/java.lang.Thread.run(Thread.java:1570)
Exception in thread "Thread-1" java.lang.RuntimeException: Sequence and quality length don't match: 'GGGGGGGGGGGGGGGGGGGGGTGGGGTAGGGTTTGTATAGTGGTTGGGGTGGGGGGAGTATTGGTTGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGTCACACATAAGCAACATTTCGAAGGGTATGGTGATTTGGAATGACAAGAGTGTAAGATCGTAAGAGCGTCGTGAAGGGAAAGA' vs ',:FF:F,:FF,FFFFF:FFFFF,:FFFF,FF,,FF::,,:F:,:FF::,FFFF,F:FF:,F:FFFFFFF,F,FFFFF,,FFF:FF,::FF,FF,:F:F:F,FFFF,F,FFFFFFF,::F,FFFFF,F,,FFF:F,:,FFFF,F,,,FFFFF'
	at org.usadellab.trimmomatic.fastq.FastqRecord.<init>(FastqRecord.java:25)
	at org.usadellab.trimmomatic.fastq.FastqParser.parseOne(FastqParser.java:89)
	at org.usadellab.trimmomatic.fastq.FastqParser.next(FastqParser.java:179)
	at org.usadellab.trimmomatic.threading.ParserWorker.run(ParserWorker.java:42)
	at java.base/java.lang.Thread.run(Thread.java:1570)

##### Debugging
bioawk -cfastx 'length($seq) != length($qual)' RAWDATA/SRR23378605_1.fq.gz |wc
- output: 0    0    0
- The lengths are not different, so look into the file 


(FastqDump) [smomw681@nesh-login3 DATA]$ zgrep -A 3 "AAAGGCATCGGTTGTCAGGTTCGTTGCTATGGGTGTACGTAACCATTTCAATTTTTCGGGGCGTAGGGCTTAGTTTCCACGCGTTCAATGAGAGCAGACCACCGTGAAGAGCTTTGGAATAGCACTTCAGTGGCTTCTGGCCATGACCTCC" RAWDATA/SRR23378605_1.fastq.gz
AAAGGCATCGGTTGTCAGGTTCGTTGCTATGGGTGTACGTAACCATTTCAATTTTTCGGGGCGTAGGGCTTAGTTTCCACGCGTTCAATGAGAGCAGACCACCGTGAAGAGCTTTGGAATAGCACTTCAGTGGCTTCTGGCCATGACCTCC
+SRR23378605.25279883 HWI-A00245_BSF_0734:2:2119:9064:32503 length=151
F:FFFFF,FFF,F:FFFFF,FFFFFFFFFFFFFFFFFFFFFFFF::F:FFFFFFF:FFFFFFFFFFFFF:FFF,FFFFFFFFFFFFFFFFFFFFFF,FFF:F:F,FFFF,:F:

(FastqDump) [smomw681@nesh-login3 DATA]$ zgrep -A 3 "GGGGGGGGGGGGGGGGGGGGGTGGGGTAGGGTTTGTATAGTGGTTGGGGTGGGGGGAGTATTGGTTGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGTCACACATAAGCAACATTTCGAAGGGTATGGTGATTTGGAATGACAAGAGTGTAAGATCGTAAGAGCGTCGTGAAGGGAAAGA" RAWDATA/SRR23378605_1.fastq.gz
gzip: RAWDATA/SRR23378605_1.fastq.gz: invalid compressed data--format violated

- Conclusion: the raw fastq.gz file was corrupt
	- The files are downloaded anew and the problem was solved
